In [ ]:
import os
from spatialdata import read_zarr
import sparrow as sp

In [ ]:
import scanpy as sc
import squidpy as sq
#sdata_merged.write("/Users/Xenium/All_samples_together/merged/sdata_merged_annotated.zarr" )
sdata_merged=read_zarr( "/Users/Xenium/All_samples_together/merged/sdata_merged_annotated.zarr" )

In [ ]:
old_to_new ={'0':'Melanoma',
'1':'Melanoma',
'2':'Melanoma',
'3':'Melanoma',
'4':'Melanoma',
'5':'Melanoma',
'6':'Melanoma',
'7':'Melanoma',
'8':'Melanoma',
'9':'Melanoma',
'10':'Melanoma',
'11':'CD8Tcells',#GZMA
'12':'Melanoma',#HLA melanoma_AP
'13':'Melanoma',
'14':'Macrophages',
'15':'Macrophages',#Monocytes?
'16':'CD4Tcells',
'17':'Melanoma',
'18':'Melanoma',
'19':'Melanoma',
'20':'CD8Tcells', #activated
'21':'Blood_endothelial_cells',
'22':'Melanoma',
'23':'Melanoma',
'24':'Melanoma',
'25':'Melanoma',#HLA melanoma_AP',
'26':'CAFs',
'27':'Pericytes',
'28':'Melanoma',
'29':'Melanoma',
'30':'UNKNOWN',
'31':'Macrophages',#Neutro?
'32':'Melanoma',
'33':'Melanoma',
'34':'Melanoma',
'35':'CAFs',
'36':'CAFs',
'37':'UNKNOWN',
'38':'UNKNOWN',
'39':'CAFs',
'40':'Tregs',
'41':'CAFs',
'42':'UNKNOWN',
'43':'Keratinocytes',
'44':'CAFs',
'45':'CD8Tcells',#prolif
'46':'Macrophages', #_ITGAX
'47':'Bcells',
'48':'Macrophages', #_MRC1
'49':'CAFs',
'50':'cDC1',
'51':'cDC2',
'52':'Melanoma',
'53':'Macrophages',
'54':'NKcells',
'55':'Melanoma',
'56':'Lymphatic_endothelial_cells'}
sdata_merged.table.obs['annotated_1'] = (
    sdata_merged.table.obs['leiden']
    .map(old_to_new)
    .astype('category')
)

sc.settings.figdir = "/Users/Library/CloudStorage/OneDrive-KULeuven/Documents/PROJECTS/NK_PAPER/SUBMISSION/SUBMISSION_2/FIGURES_AI/Figure_4_raws/"

sc.pl.umap(sdata_merged.table, color=['annotated_1'],size=0, alpha=0, save= "UMAP_annotated.svg")

# Define new categories for each label
new_category_mapping = {
    'GC16_BT_NR': 'Before_treatment_excluded',
    'GC16_OT_NR': 'On_treatment_excluded',
    'GC17_OT_NR': 'On_treatment_excluded',
    'GC18_OT_NR': 'On_treatment_excluded',
    'GC19_BT_NR': 'Before_treatment_cold',
    'GC19_OT_NR': 'On_treatment_cold',
    'GC26_OT_NR': 'On_treatment_excluded',
    'GC36_OT_NR': 'On_treatment_excluded'
}

# Apply the new categories to the AnnData table
sdata_merged.table.obs['timepoint'] = sdata_merged.table.obs['fov_labels'].map(new_category_mapping)

marker_genes = ["AREG","FCER1G","FCGR3A","GNLY","KLRB1","KLRC1","KLRF1","KLRK1","LAMP1","NCAM1","TRDC","TYROBP","XCL1","NCR1","NKG7","XCL2", "PDCD1", "CD274", "CD69",]#"PDCD1","CASP3","CASP7","CCL3","CCL4","CCL5","CD274","CD40","CD40LG","CXCL10","CXCL13","CXCR3","EOMES","FAS","FASLG","FOS","GZMA","GZMB","GZMH","GZMK","ICAM1","ICAM2","IFNAR1","IFNAR2","IFNG","IFNGR1","IFNGR2","IL10","IL12A","IL15","IL2RA","IL6","IL7R","ITGAE","LAG3","MKI67","NLRC5","PRF1","PTPRC","SELL","TCF7","TIGIT","VEGFA","HAVCR2","SELL","XCR1"]
genes2 = ["GZMB","GNLY", "PRF1", "TYROBP", "XCL1", "XCL2","CCL3","CCL5", "IFNG","RGS1"]
genes_marker_scRNA = ["PTPRC","SOX10","NLRC5","CIITA", "HLA-B", "HLA-DQA1","CD8A","MKI67","IFNG","IL7R", "AREG", "SELL", "TIGIT", "TYROBP","FCGR3A","TRDC","KLRB1","KLRF1", "KLRC1","NCAM1", "KLRK1", "NCR1","NKG7","IFNG","GNLY","GZMB", "PRF1","XCL1","XCL2","CCL3", "CCL4","CCL5", "IL10", "IL15",  "PDCD1", "CD274", "CD69", "NR4A2", "CLEC10A", "CLEC9A","KRT15", "PDGFRA", "LYVE1", "CD34"]
NK_only_genes =["AREG", "SELL", "TIGIT", "TYROBP","FCGR3A","TRDC","KLRB1","KLRF1", "KLRC1","NCAM1", "KLRK1", "NCR1","NKG7","IFNG","GNLY","GZMB", "PRF1","XCL1","XCL2","CCL3", "CCL4","CCL5", "CD8A"]

sdata_merged.table_NK = sdata_merged.table[sdata_merged.table.obs.annotated_1 == 'NKcells']
import matplotlib.pyplot as plt

# Generate the dot plot
dotplot = sc.pl.dotplot(
    sdata_merged.table_NK,
    NK_only_genes,
    groupby=['annotated_1', 'timepoint'],
    dendrogram=False,
    # dot_min=0.1,
    standard_scale='var',
    figsize=(8, 2),
    show=False  # Prevents immediate display
)

# Access the figure and main plot axis
fig = plt.gcf()  # Get the current figure
ax = fig.axes[0]  # Main plot axis is typically the first axis

# Get the number of groups
combined_groups = sdata_merged.table_NK.obs.groupby(['annotated_1', 'timepoint']).size().index.tolist()
num_groups = len(combined_groups)

# Add horizontal lines between the combined groups
for i in range(1, num_groups):  # Skip the first one; we don't need a line at the top
    ax.axhline(y=i - 0.5, color='gray', linestyle='--', linewidth=0.5)

# Add an extra line at the bottom of the last row of dots
ax.axhline(y=num_groups - 0.5, color='gray', linestyle='--', linewidth=0.5)
fig.set_size_inches(8, 3)  # Set width and height in inches
plt.rcParams['pdf.fonttype'] = 42  # Type 42 (TrueType) keeps text editable
plt.rcParams['font.family'] = 'Arial'
plt.subplots_adjust(left=0.2, right=0.95, top=0.9, bottom=0.32)

plt.savefig("/Users/Library/CloudStorage/OneDrive-KULeuven/Documents/PROJECTS/NK_PAPER/SUBMISSION/SUBMISSION_2/FIGURES_AI/Figure_4_raws/Dotplot_NK_with_lines.pdf", format='pdf')

# Display the modified plot
plt.show()



import matplotlib.pyplot as plt

# Generate the dot plot
dotplot = sc.pl.dotplot(
    sdata_merged.table,
    genes_marker_scRNA,
    groupby=['annotated_1'],
    dendrogram=False,
    # dot_min=0.1,
    standard_scale='var',
    figsize=(13, 5),
    show=False  # Prevents immediate display
)

# Access the axes directly from Matplotlib's current figure
fig = plt.gcf()  # Get the current figure
ax = fig.axes[0]  # Typically, the first axis is the main plot

# Add horizontal lines between cell types
cell_types = sdata_merged.table.obs['annotated_1'].unique()  # Get unique cell types
num_cell_types = len(cell_types)

# Iterate and add horizontal lines
for i in range(1, num_cell_types):
    ax.axhline(y=i - 0.5, color='gray', linestyle='--', linewidth=0.5)

# Add an extra line at the bottom of the last row of dots
ax.axhline(y=num_cell_types - 0.5, color='gray', linestyle='--', linewidth=0.5)
fig.set_size_inches(13, 6)  # Set width and height in inches
plt.rcParams['pdf.fonttype'] = 42  # Type 42 (TrueType) keeps text editable
plt.rcParams['font.family'] = 'Arial'
plt.subplots_adjust(left=0.2, right=0.95, top=0.9, bottom=0.32)

plt.savefig("/Users/u0128760/Library/CloudStorage/OneDrive-KULeuven/Documents/PROJECTS/NK_PAPER/SUBMISSION/SUBMISSION_2/FIGURES_AI/Figure_4_raws/Dotplot_with_lines.pdf", format='pdf')
# Display the modified plot
plt.show()


In [ ]:
import os
import matplotlib.pyplot as plt

# Set the directory where the files will be saved
output_dir = '/Users/u0128760/Library/CloudStorage/OneDrive-KULeuven/Documents/PROJECTS/NK_PAPER/SUBMISSION/SUBMISSION_2/FIGURES_AI/Figure_4_raws/'  # Change this to your desired path
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Filter the data as per the provided conditions
sdata_merged.table_subset = sdata_merged.table[sdata_merged.table.obs.annotated_1 != 'UNKNOWN']
sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'Keratinocytes']
# Uncomment the following lines to exclude more annotations if needed
# sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'Bcells']
# sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'CD8Tcells_activated']
# sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'CD8Tcells_prolif']
sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'Melanoma_AP']
sdata_merged.table_subset = sdata_merged.table_subset[sdata_merged.table_subset.obs.annotated_1 != 'Melanoma']

# Add new annotation column
sdata_merged.table_subset.obs['annotated_1_new'] = sdata_merged.table_subset.obs['annotated_1']

# Get unique timepoints
timepoints = sdata_merged.table_subset.obs['timepoint'].unique()

# Loop through each timepoint, recompute spatial neighbors, and plot heatmap
for timepoint in timepoints:
    # Filter for each timepoint
    sdata_timepoint = sdata_merged.table_subset[sdata_merged.table_subset.obs['timepoint'] == timepoint]
    
    # Count cells per group in "annotated_1_new"
    group_counts = sdata_timepoint.obs['annotated_1_new'].value_counts()

    # Identify groups with less than 10 cells
    groups_to_keep = group_counts[group_counts >= 10].index.tolist()
    
    # Filter out cells belonging to groups with less than 10 cells
    sdata_timepoint = sdata_timepoint[sdata_timepoint.obs['annotated_1_new'].isin(groups_to_keep)]
    
    # Recompute spatial neighbors for the specific timepoint
    sq.gr.spatial_neighbors(sdata_timepoint)
    
    # Compute neighborhood enrichment for the specific timepoint
    sq.gr.nhood_enrichment(sdata_timepoint, cluster_key='annotated_1_new')
    
    # Plot neighborhood enrichment heatmap for each timepoint
    fig = sq.pl.nhood_enrichment(
        sdata_timepoint, 
        cluster_key="annotated_1_new", 
        method="ward",
        figsize=(7, 7),
        title=f"Nhood enrichment heatmap - Timepoint {timepoint}",
        robust=True,
        cmap="seismic"
    )
    
    # Create the full file path
    pdf_filename = os.path.join(output_dir, f"nhood_enrichment_heatmap_timepoint_{timepoint}.pdf")
    
    # Save the heatmap as a PDF file
    plt.savefig(pdf_filename, format='pdf', dpi=150, bbox_inches='tight')
    
    # Close the figure to free memory
    plt.close()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors

# List of desired cell types
sdata_merged.table.obs.index = sdata_merged.table.obs.index.astype(str) + "_" + sdata_merged.table.obs['fov_labels'].astype(str)

# Step 1: Filter CAF cells
caf_cells = sdata_merged.table[sdata_merged.table.obs['annotated_1'] == 'NKcells']
caf_cells = caf_cells[~caf_cells.obs.index.duplicated()]
caf_cells.obs['timepoint'] = sdata_merged.table.obs.loc[caf_cells.obs.index, 'timepoint']
sdata_merged.table.obs = sdata_merged.table.obs[~sdata_merged.table.obs.index.duplicated()]

# Step 2: Get spatial coordinates for all cells and subset for CAFs
spatial_coords = sdata_merged.table.obsm['spatial']
caf_coords = spatial_coords[sdata_merged.table.obs['annotated_1'] == 'NKcells']

# Step 3: Initialize NearestNeighbors model using spatial coordinates
nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(spatial_coords)  # 11 to include the CAF itself
 
# Step 4: Find the 10 nearest neighbors (excluding the CAF itself) for each CAF cell
distances, indices = nbrs.kneighbors(caf_coords)
 
# Remove the first column to exclude each CAF itself from its own neighborhood
neighbor_indices = indices[:, 1:]
 
# Step 5: Get the neighboring cell types and prepare a DataFrame for calculating proportions
neighbor_types = sdata_merged.table.obs['annotated_1'].values[neighbor_indices]
neighbor_types_df = pd.DataFrame(neighbor_types, columns=[f'Neighbor_{i+1}' for i in range(10)], index=caf_cells.obs.index)
 
# Step 6: Calculate proportions for each cell type in the 10 nearest neighbors for each CAF cell
neighbor_type_counts = neighbor_types_df.apply(pd.Series.value_counts, axis=1).fillna(0)
neighbor_type_proportions = neighbor_type_counts.div(neighbor_type_counts.sum(axis=1), axis=0)
 
# Step 7: Add condition information to CAF cells
caf_cells.obs['timepoint'] = sdata_merged.table.obs.loc[caf_cells.obs.index, 'timepoint']
 
# Step 8: Calculate the mean proportion for each cell type across CAFs within each condition
mean_proportions_by_condition = neighbor_type_proportions.groupby(caf_cells.obs['timepoint']).mean()
# Step 1: Calculate the mean proportion for each cell type, grouped by 'timepoint' and 'fov_labels'
mean_proportions_by_condition_and_fov = neighbor_type_proportions.groupby(
    [caf_cells.obs['timepoint'], caf_cells.obs['fov_labels']]
).mean()

# Step 2: Reset index to organize data for plotting
mean_proportions_by_condition_and_fov.reset_index(inplace=True)

# Step 3: Pivot the data for easier plotting
proportions_pivot = mean_proportions_by_condition_and_fov.pivot_table(
    index=['timepoint', 'fov_labels'], 
    values=mean_proportions_by_condition_and_fov.columns[2:],  # Assuming first 2 are 'timepoint' and 'fov_labels'
    aggfunc='mean'
)

# Step 4: Plot stacked bar chart grouped by 'timepoint' and 'fov_labels'
fig, ax = plt.subplots(figsize=(12, 8))

proportions_pivot.plot(
    kind='bar', 
    stacked=True, 
    colormap='tab20', 
    ax=ax, 
    width=0.8
)

# Customize x-axis labels to reflect both 'timepoint' and 'fov_labels'
ax.set_xticklabels(
    [f"{timepoint} - {fov}" for timepoint, fov in proportions_pivot.index], 
    rotation=45, 
    ha='right'
)

# Add plot labels and legend
plt.title('Average Proportion of Neighbor Cell Types for NKcells', fontsize=14)
plt.xlabel('Timepoint - FOV Labels', fontsize=12)
plt.ylabel('Proportion of Neighbor Cell Types', fontsize=12)
plt.legend(title='Cell Types', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
plt.tight_layout()

# Show the plot
plt.show()



In [ ]:
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import pandas as pd  # Assuming pandas is already imported

# Ensuring unique indices in the obs DataFrame
sdata_merged.table.obs.index = (
    sdata_merged.table.obs.index.astype(str) + "_" + sdata_merged.table.obs['fov_labels'].astype(str)
)

# List of desired cell types
cell_types_to_include = ["NKcells", "CD4Tcells", "CD8Tcells", "cDC1", "cDC2", "Tregs", "Bcells", "Macrophages"]

# Subset the AnnData object based on the 'annotated_1' column
sdata_subset = sdata_merged.table[sdata_merged.table.obs['annotated_1'].isin(cell_types_to_include)]

# Step 1: Filter NKcells
nk_cells = sdata_subset[sdata_subset.obs['annotated_1'] == 'NKcells']
nk_cells = nk_cells[~nk_cells.obs.index.duplicated()]
nk_cells.obs['timepoint'] = sdata_subset.obs.loc[nk_cells.obs.index, 'timepoint']

sdata_subset.obs = sdata_subset.obs[~sdata_subset.obs.index.duplicated()]

# Step 2: Get spatial coordinates for all cells and subset for NKcells
spatial_coords = sdata_subset.obsm['spatial']
nk_coords = spatial_coords[sdata_subset.obs['annotated_1'] == 'NKcells']

# Step 3: Initialize NearestNeighbors model
nbrs = NearestNeighbors(n_neighbors=21, algorithm='ball_tree').fit(spatial_coords)

# Step 4: Find the 5 nearest neighbors (excluding the NK cell itself)
distances, indices = nbrs.kneighbors(nk_coords)
neighbor_indices = indices[:, 1:]

# Step 5: Get neighboring cell types and calculate proportions
neighbor_types = sdata_subset.obs['annotated_1'].values[neighbor_indices]
neighbor_types_df = pd.DataFrame(neighbor_types, columns=[f'Neighbor_{i+1}' for i in range(20)], index=nk_cells.obs.index)

neighbor_type_counts = neighbor_types_df.apply(pd.Series.value_counts, axis=1).fillna(0)
neighbor_type_proportions = neighbor_type_counts.div(neighbor_type_counts.sum(axis=1), axis=0)

# Step 6: Add timepoint information to NKcells
nk_cells.obs['timepoint'] = sdata_subset.obs.loc[nk_cells.obs.index, 'timepoint']

# Step 7: Group by 'timepoint' and calculate mean proportions
mean_proportions_by_timepoint = neighbor_type_proportions.groupby(nk_cells.obs['timepoint']).mean()

# Step 8: Prepare data for plotting
mean_proportions_by_timepoint.reset_index(inplace=True)

# Step 9: Filter only specific timepoints
filtered_timepoints = ["Before_treatment_excluded", "On_treatment_excluded"]
mean_proportions_filtered = mean_proportions_by_timepoint[
    mean_proportions_by_timepoint['timepoint'].isin(filtered_timepoints)
]

# Step 10: Plot stacked bar chart grouped by filtered timepoints
fig, ax = plt.subplots(figsize=(10, 6))

mean_proportions_filtered.set_index('timepoint').plot(
    kind='bar', 
    stacked=True, 
    colormap='tab20', 
    ax=ax, 
    width=0.6  # Adjusted width for better appearance
)

# Customize x-axis labels and aesthetics
ax.set_xticklabels(mean_proportions_filtered['timepoint'], rotation=45, ha='right')
plt.title('Average Proportion of Neighbor Cell Types for NKcells by Selected Timepoints', fontsize=14)
plt.xlabel('Timepoint', fontsize=12)
plt.ylabel('Proportion of Neighbor Cell Types', fontsize=12)
plt.legend(
    title='Cell Types', 
    bbox_to_anchor=(1.05, 1), 
    loc='upper left', 
    fontsize=10,
    title_fontsize=11
)
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add gridlines for better readability
plt.tight_layout(pad=2.0)  # Adjust padding for better layout

# Set the directory to save the plots
output_dir = 'output/plots'
os.makedirs(output_dir, exist_ok=True)

# Save the plot as PDF and PNG
plt.savefig("/Users/u0128760/Library/CloudStorage/OneDrive-KULeuven/Documents/PROJECTS/NK_PAPER/SUBMISSION/SUBMISSION_2/FIGURES_AI/Figure_4_raws/Stacked_bars.pdf", format='pdf')

# Show the plot
plt.show()

# Close the figure to free memory
plt.close()
